<a href="https://colab.research.google.com/github/guillaumetougas/sbl_hyperspectral_ai/blob/main/02_scripts/sbl_hyperspec_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1D-CNN model from hyperspectral data


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch
import torchvision as tv
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchaudio as ta
!pip install rasterio
import rasterio as rio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 94.7 MB/s eta 0:00:00


###Load preprocessed HS data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
#Load HS data with geopandas
smoothed_data=gpd.read_file("/content/drive/MyDrive/2024-Tougas-PhD/Data_SBL_HS_AI/sbl_smoothed_spectra.gpkg")

#isolate spectral data and prepare with rasterio
spectral_data_labeled = smoothed_data.drop(columns=['geometry']).values
print(np.shape(spectral_data_labeled))
print(spectral_data_labeled[:5])

spectral_data = spectral_data_labeled[:, 1:242]

# Normalization (recommended for pretrained genereic models like faster rcnn resnet 50)
spectral_data = (spectral_data - np.min(spectral_data)) / (np.max(spectral_data) - np.min(spectral_data))

# Ensure spectral_data has a numeric dtype before converting to tensor
spectral_data = spectral_data.astype(np.float32)  # or np.float64

# Conversion in PyTorch tensor
spectral_tensor = torch.tensor(spectral_data, dtype=torch.float32)

print(spectral_tensor.shape)

labels = smoothed_data['Label']
print(labels)

(21696, 246)
[['1' 0.0014360744236254599 0.0016860739034256812 ... 'BEPA'
  20.6998377058 18.0239535584]
 ['10' 0.0 3.7468307932240064e-06 ... 'ACRU' 8.92277349821 3.50901934984]
 ['100' 0.0 0.00018155946172550302 ... 'Picea' 12.975775825 7.35715668545]
 ['1000' 0.00011515454612388588 0.000444991645046132 ... 'ACSA'
  12.9757616982 8.00540072982]
 ['10000' 0.005083690612008917 0.004094856722796195 ... 'ABBA'
  9.26431133742 5.2001091841]]
torch.Size([21696, 241])
0         BEPA
1         ACRU
2        Picea
3         ACSA
4         ABBA
         ...  
21691     THOC
21692     THOC
21693    Picea
21694    Picea
21695     Mort
Name: Label, Length: 21696, dtype: object


### Create the 1D-CNN model

In [5]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Example hyperspectral data (N samples, C spectral bands)
# Replace this with your actual data
N, C = 21696, 242  # Example: 1000 pixels, 242 bands
X = torch.rand(N, C)  # Random data for demonstration
y = torch.randint(0, 10, (N,))  # Random labels (10 classes)

# Normalize the data
X = (X - X.min()) / (X.max() - X.min())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Define a 1D CNN for hyperspectral data classification
class HyperspectralCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(HyperspectralCNN, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * (input_channels // 2), 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

###Prepare the dataset and data loaders



In [ ]:
# Create a custom dataset class for your hyperspectral data and annotations
# ...

# Create data loaders for training and validation
# ...

###Train the model

In [ ]:
# Define an optimizer and loss function
# ...

# Train the model for a certain number of epochs
# ...

###Evaluate the model

In [ ]:
# Evaluate the model on a test dataset
# ...